# Fine-tune Gemma 7B Comparison Across Agreement Levels

This notebook fine-tunes Gemma-7B using different agreement levels from FinancialPhraseBank to compare the impact of data quality on model performance.

## Agreement Levels:
- **sentences_50agree**: ≥50% annotator agreement (4,846 sentences)
- **sentences_66agree**: ≥66% annotator agreement (4,217 sentences) 
- **sentences_75agree**: ≥75% annotator agreement (3,453 sentences)
- **sentences_allagree**: 100% annotator agreement (2,264 sentences)

## Hypothesis:
Higher agreement levels should lead to better model performance due to reduced label noise.

In [1]:
# Install required packages
%pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
%pip install -q -U transformers==4.38.2
%pip install -q accelerate==0.32.0
%pip install -q -i https://pypi.org/simple/ bitsandbytes
%pip install -q -U datasets==2.16.1
%pip install -q -U trl==0.7.11
%pip install -q -U peft==0.10.0

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import gc
import time
import warnings
from datetime import datetime

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    TrainingArguments, 
    pipeline, 
    logging
)
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer

from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)
from sklearn.model_selection import train_test_split

print(f"transformers=={transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

transformers==4.38.2
PyTorch version: 2.5.1+cu121
CUDA available: True


## Define Helper Functions

In [4]:
def clear_memory():
    """Clear GPU and CPU memory"""
    print("Clearing memory...")
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"GPU memory reserved: {torch.cuda.memory_reserved()/1024**3:.2f} GB")

def wait_and_clear(seconds=60):
    """Wait for specified seconds and clear memory"""
    print(f"Waiting {seconds} seconds...")
    time.sleep(seconds)
    clear_memory()

In [5]:
def prepare_dataset(agreement_level):
    """Load and prepare dataset for specific agreement level"""
    print(f"\nLoading dataset: {agreement_level}...")
    
    # Load the dataset from HuggingFace
    dataset = load_dataset("takala/financial_phrasebank", agreement_level)
    
    # Convert to pandas for easier manipulation
    df = dataset['train'].to_pandas()
    df = df.rename(columns={'sentence': 'text', 'label': 'sentiment'})
    
    print(f"Dataset shape: {df.shape}")
    print(f"Sentiment distribution:")
    sentiment_dist = df['sentiment'].value_counts()
    print(sentiment_dist)
    
    return df, sentiment_dist

In [6]:
def create_balanced_splits(df, max_samples_per_class=300):
    """Create balanced train/val/test splits"""
    # Calculate minimum class size
    min_class_size = df['sentiment'].value_counts().min()
    samples_per_class = min(max_samples_per_class, min_class_size)
    
    # Use 70% for training, 15% for validation, 15% for testing
    train_size_per_class = int(samples_per_class * 0.7)
    val_size_per_class = int(samples_per_class * 0.15)
    test_size_per_class = samples_per_class - train_size_per_class - val_size_per_class
    
    print(f"Split sizes per class: Train={train_size_per_class}, Val={val_size_per_class}, Test={test_size_per_class}")
    
    X_train, X_val, X_test = [], [], []
    
    # Map integer labels to sentiment names
    label_mapping = {0: "negative", 1: "neutral", 2: "positive"}
    
    for sentiment_label in [0, 1, 2]:
        sentiment_name = label_mapping[sentiment_label]
        sentiment_data = df[df.sentiment == sentiment_label]
        
        if len(sentiment_data) == 0:
            print(f"Warning: No samples found for {sentiment_name} sentiment")
            continue
        
        # Sample the required number for this class
        if len(sentiment_data) >= samples_per_class:
            sampled_data = sentiment_data.sample(n=samples_per_class, random_state=42)
        else:
            sampled_data = sentiment_data
            print(f"Warning: Only {len(sentiment_data)} samples available for {sentiment_name}")
        
        # Split the sampled data
        if len(sampled_data) >= 3:  # Need at least 3 samples to split
            temp_data, test_data = train_test_split(
                sampled_data, 
                test_size=min(test_size_per_class, len(sampled_data)//3),
                random_state=42
            )
            
            if len(temp_data) >= 2:
                train_data, val_data = train_test_split(
                    temp_data,
                    test_size=min(val_size_per_class, len(temp_data)//2),
                    random_state=42
                )
            else:
                train_data = temp_data
                val_data = pd.DataFrame()
        else:
            train_data = sampled_data
            val_data = pd.DataFrame()
            test_data = pd.DataFrame()
        
        if len(train_data) > 0:
            X_train.append(train_data)
        if len(val_data) > 0:
            X_val.append(val_data)
        if len(test_data) > 0:
            X_test.append(test_data)
    
    # Combine and shuffle
    X_train = pd.concat(X_train).sample(frac=1, random_state=10).reset_index(drop=True) if X_train else pd.DataFrame()
    X_val = pd.concat(X_val).sample(frac=1, random_state=10).reset_index(drop=True) if X_val else pd.DataFrame()
    X_test = pd.concat(X_test).sample(frac=1, random_state=10).reset_index(drop=True) if X_test else pd.DataFrame()
    
    print(f"Final splits: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")
    
    return X_train, X_val, X_test

In [7]:
def prepare_prompts(X_train, X_val, X_test, tokenizer):
    """Prepare training prompts"""
    EOS_TOKEN = tokenizer.eos_token
    label_mapping = {0: "negative", 1: "neutral", 2: "positive"}
    
    def generate_prompt(data_point):
        sentiment_text = label_mapping[data_point["sentiment"]]
        return f"""Analyze the sentiment of the news headline enclosed in square brackets, 
                determine if it is positive, neutral, or negative, and return the answer as 
                the corresponding sentiment label "positive" or "neutral" or "negative"

                [{data_point["text"]}] = {sentiment_text}
                """.strip() + EOS_TOKEN

    def generate_test_prompt(data_point):
        return f"""Analyze the sentiment of the news headline enclosed in square brackets, 
                determine if it is positive, neutral, or negative, and return the answer as 
                the corresponding sentiment label "positive" or "neutral" or "negative"

                [{data_point["text"]}] = 
                """.strip()
    
    # Generate prompts
    train_prompts = pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=["text"])
    val_prompts = pd.DataFrame(X_val.apply(generate_prompt, axis=1), columns=["text"])
    
    # Test prompts and true labels
    y_true = [label_mapping[label] for label in X_test['sentiment'].tolist()]
    test_prompts = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])
    
    # Convert to HuggingFace datasets
    train_data = Dataset.from_pandas(train_prompts)
    eval_data = Dataset.from_pandas(val_prompts)
    
    return train_data, eval_data, test_prompts, y_true

In [8]:
def load_model_and_tokenizer():
    """Load fresh model and tokenizer"""
    model_name = "google/gemma-7b"
    
    compute_dtype = getattr(torch, "float16")
    
    # Configure 4-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
    )
    
    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        quantization_config=bnb_config, 
    )
    
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    
    # Load tokenizer
    max_seq_length = 2048
    tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)
    
    return model, tokenizer

In [9]:
def evaluate_model(y_true, y_pred, agreement_level):
    """Comprehensive evaluation function"""
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none': 1, 'negative': 0}
    
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)
    
    # Calculate overall accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    
    print(f"\n{'='*60}")
    print(f"RESULTS FOR {agreement_level.upper()}")
    print(f"{'='*60}")
    print(f'Overall Accuracy: {accuracy:.3f}')
    
    # Calculate per-class accuracy
    unique_labels = set(y_true_mapped)
    
    class_accuracies = {}
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) 
                         if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        label_name = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}[label]
        class_accuracies[label_name] = label_accuracy
        print(f'{label_name} Accuracy: {label_accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(
        y_true=y_true_mapped, 
        y_pred=y_pred_mapped,
        target_names=['Negative', 'Neutral', 'Positive'],
        output_dict=True
    )
    
    print('\nClassification Report:')
    print(classification_report(
        y_true=y_true_mapped, 
        y_pred=y_pred_mapped,
        target_names=['Negative', 'Neutral', 'Positive']
    ))
    
    return {
        'agreement_level': agreement_level,
        'overall_accuracy': accuracy,
        'class_accuracies': class_accuracies,
        'classification_report': class_report,
        'test_samples': len(y_true)
    }

In [10]:
def predict_sentiment(test_prompts, model, tokenizer):
    """Prediction function"""
    y_pred = []
    for i in tqdm(range(len(test_prompts)), desc="Predicting"):
        prompt = test_prompts.iloc[i]["text"]
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        
        with torch.no_grad():
            outputs = model.generate(
                **input_ids, 
                max_new_tokens=1, 
                temperature=0.0,
                do_sample=False
            )
        
        result = tokenizer.decode(outputs[0])
        answer = result.split("=")[-1].lower().strip()
        
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    
    return y_pred

In [11]:
def fine_tune_model(model, tokenizer, train_data, eval_data, agreement_level):
    """Fine-tune model for specific agreement level"""
    # LoRA configuration
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
    )
    
    # Training arguments
    training_steps = len(train_data) // 8
    eval_steps = max(training_steps // 10, 10)
    
    training_arguments = TrainingArguments(
        output_dir=f"logs_{agreement_level}",
        num_train_epochs=3,  # Reduced epochs for comparison
        gradient_checkpointing=True,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        optim="paged_adamw_32bit",
        save_steps=0,
        logging_steps=max(training_steps // 20, 5),
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=True,
        bf16=False,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_ratio=0.03,
        group_by_length=False,
        evaluation_strategy='steps',
        eval_steps=eval_steps,
        eval_accumulation_steps=1,
        lr_scheduler_type="cosine",
        report_to="tensorboard",
    )
    
    # Initialize trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=eval_data,
        peft_config=peft_config,
        dataset_text_field="text",
        tokenizer=tokenizer,
        max_seq_length=2048,
        args=training_arguments,
        packing=False,
    )
    
    print(f"\nStarting fine-tuning for {agreement_level}...")
    print(f"Training samples: {len(train_data)}")
    print(f"Validation samples: {len(eval_data)}")
    
    # Start training
    trainer.train()
    
    # Save model
    model_save_path = f"../models/trained-gemma-{agreement_level}"
    trainer.model.save_pretrained(model_save_path)
    print(f"Model saved to: {model_save_path}")
    
    return trainer.model

## Main Comparison Loop

In [12]:
# Define agreement levels to test
agreement_levels = [
    "sentences_50agree",
    "sentences_66agree", 
    "sentences_75agree",
    "sentences_allagree"
]

# Store results
all_results = []
detailed_results = {}

print(f"Starting comparison across {len(agreement_levels)} agreement levels...")
print(f"Agreement levels: {agreement_levels}")

# Clear initial memory
clear_memory()

Starting comparison across 4 agreement levels...
Agreement levels: ['sentences_50agree', 'sentences_66agree', 'sentences_75agree', 'sentences_allagree']
Clearing memory...
GPU memory allocated: 0.00 GB
GPU memory reserved: 0.00 GB


In [13]:
for i, agreement_level in enumerate(agreement_levels):
    print(f"\n{'='*80}")
    print(f"PROCESSING {agreement_level.upper()} ({i+1}/{len(agreement_levels)})")
    print(f"{'='*80}")
    
    try:
        # Step 1: Prepare dataset
        df, sentiment_dist = prepare_dataset(agreement_level)
        
        # Step 2: Create splits
        X_train, X_val, X_test = create_balanced_splits(df)
        
        if len(X_train) == 0 or len(X_test) == 0:
            print(f"Skipping {agreement_level} - insufficient data")
            continue
        
        # Step 3: Load fresh model and tokenizer
        print("\nLoading fresh model and tokenizer...")
        model, tokenizer = load_model_and_tokenizer()
        
        # Step 4: Prepare prompts
        train_data, eval_data, test_prompts, y_true = prepare_prompts(X_train, X_val, X_test, tokenizer)
        
        # Step 5: Test baseline performance (quick test on subset)
        print("\nTesting baseline performance...")
        test_subset_size = min(20, len(test_prompts))  # Small subset for baseline
        test_subset = test_prompts.head(test_subset_size)
        true_subset = y_true[:test_subset_size]
        
        baseline_predictions = predict_sentiment(test_subset, model, tokenizer)
        baseline_accuracy = accuracy_score(
            np.vectorize(lambda x: {'positive': 2, 'neutral': 1, 'none': 1, 'negative': 0}.get(x, 1))(true_subset),
            np.vectorize(lambda x: {'positive': 2, 'neutral': 1, 'none': 1, 'negative': 0}.get(x, 1))(baseline_predictions)
        )
        print(f"Baseline accuracy on {test_subset_size} samples: {baseline_accuracy:.3f}")
        
        # Step 6: Fine-tune model
        fine_tuned_model = fine_tune_model(model, tokenizer, train_data, eval_data, agreement_level)
        
        # Step 7: Evaluate fine-tuned model
        print(f"\nEvaluating fine-tuned model on {len(test_prompts)} test samples...")
        final_predictions = predict_sentiment(test_prompts, fine_tuned_model, tokenizer)
        
        # Step 8: Calculate metrics
        results = evaluate_model(y_true, final_predictions, agreement_level)
        results['baseline_accuracy'] = baseline_accuracy
        results['dataset_size'] = len(df)
        results['train_size'] = len(train_data)
        results['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Store results
        all_results.append(results)
        detailed_results[agreement_level] = {
            'y_true': y_true,
            'y_pred': final_predictions,
            'test_data': X_test.copy()
        }
        
        print(f"\n✅ Completed {agreement_level}")
        print(f"Final accuracy: {results['overall_accuracy']:.3f}")
        print(f"Improvement over baseline: {results['overall_accuracy'] - baseline_accuracy:.3f}")
        
    except Exception as e:
        print(f"❌ Error processing {agreement_level}: {str(e)}")
        continue
    
    finally:
        # Clean up memory after each model
        if 'model' in locals():
            del model
        if 'tokenizer' in locals():
            del tokenizer
        if 'fine_tuned_model' in locals():
            del fine_tuned_model
        if 'trainer' in locals():
            del trainer
        
        # Wait and clear memory between runs (except for the last one)
        if i < len(agreement_levels) - 1:
            wait_and_clear(60)

print(f"\n🎉 Completed all {len(all_results)} agreement levels!")


PROCESSING SENTENCES_50AGREE (1/4)

Loading dataset: sentences_50agree...
Dataset shape: (4846, 2)
Sentiment distribution:
sentiment
1    2879
2    1363
0     604
Name: count, dtype: int64
Split sizes per class: Train=210, Val=45, Test=45
Final splits: Train=630, Val=135, Test=135

Loading fresh model and tokenizer...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


Testing baseline performance...


Predicting: 100%|██████████| 20/20 [00:04<00:00,  4.16it/s]


Baseline accuracy on 20 samples: 0.600


...

{'eval_loss': 3.226066827774048, 'eval_runtime': 297.9706, 'eval_samples_per_second': 0.453, 'eval_steps_per_second': 0.057, 'epoch': 2.79}
{'loss': 0.3318, 'grad_norm': 0.8214324116706848, 'learning_rate': 7.815762505632096e-07, 'epoch': 2.86}
{'loss': 0.3187, 'grad_norm': 0.7478442192077637, 'learning_rate': 1.545471346164007e-07, 'epoch': 2.92}


  0%|          | 0/17 [00:00<?, ?it/s]

{'eval_loss': 3.22692608833313, 'eval_runtime': 298.3111, 'eval_samples_per_second': 0.453, 'eval_steps_per_second': 0.057, 'epoch': 2.92}
{'train_runtime': 15764.6666, 'train_samples_per_second': 0.12, 'train_steps_per_second': 0.015, 'train_loss': 0.6225597960317236, 'epoch': 2.97}
Model saved to: ../models/trained-gemma-sentences_allagree

Evaluating fine-tuned model on 135 test samples...


Predicting: 100%|██████████| 135/135 [04:13<00:00,  1.88s/it]


RESULTS FOR SENTENCES_ALLAGREE
Overall Accuracy: 0.978
Negative Accuracy: 1.000
Neutral Accuracy: 0.956
Positive Accuracy: 0.978

Classification Report:
              precision    recall  f1-score   support

    Negative       0.96      1.00      0.98        45
     Neutral       1.00      0.96      0.98        45
    Positive       0.98      0.98      0.98        45

    accuracy                           0.98       135
   macro avg       0.98      0.98      0.98       135
weighted avg       0.98      0.98      0.98       135


✅ Completed sentences_allagree
Final accuracy: 0.978
Improvement over baseline: 0.328

🎉 Completed all 4 agreement levels!


## Results Analysis and Comparison

In [14]:
# Create comprehensive results DataFrame
if all_results:
    comparison_df = pd.DataFrame([
        {
            'Agreement_Level': result['agreement_level'],
            'Dataset_Size': result['dataset_size'],
            'Train_Size': result['train_size'],
            'Test_Size': result['test_samples'],
            'Baseline_Accuracy': result['baseline_accuracy'],
            'Final_Accuracy': result['overall_accuracy'],
            'Improvement': result['overall_accuracy'] - result['baseline_accuracy'],
            'Improvement_Percent': ((result['overall_accuracy'] - result['baseline_accuracy']) / result['baseline_accuracy']) * 100,
            'Timestamp': result['timestamp']
        }
        for result in all_results
    ])
    
    # Sort by agreement level for better display
    level_order = ['sentences_50agree', 'sentences_66agree', 'sentences_75agree', 'sentences_allagree']
    comparison_df['Level_Order'] = comparison_df['Agreement_Level'].apply(lambda x: level_order.index(x) if x in level_order else 999)
    comparison_df = comparison_df.sort_values('Level_Order').drop('Level_Order', axis=1)
    
    print("\n" + "="*100)
    print("COMPREHENSIVE RESULTS COMPARISON")
    print("="*100)
    
    # Display results table
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print(comparison_df.round(3))
    
    # Save results
    results_filename = "../results/gemma_agreement_levels_comparison.csv"
    comparison_df.to_csv(results_filename, index=False)
    print(f"\n📊 Results saved to: {results_filename}")
    
else:
    print("❌ No results to display - all experiments failed")


COMPREHENSIVE RESULTS COMPARISON
      Agreement_Level  Dataset_Size  Train_Size  Test_Size  Baseline_Accuracy  \
0   sentences_50agree          4846         630        135               0.60   
1   sentences_66agree          4217         630        135               0.70   
2   sentences_75agree          3453         630        135               0.60   
3  sentences_allagree          2264         630        135               0.65   

   Final_Accuracy  Improvement  Improvement_Percent            Timestamp  
0           0.859        0.259               43.210  2025-06-27 08:32:18  
1           0.896        0.196               28.042  2025-06-27 10:37:32  
2           0.956        0.356               59.259  2025-06-28 09:28:01  
3           0.978        0.328               50.427  2025-06-28 13:57:57  

📊 Results saved to: ../results/gemma_agreement_levels_comparison.csv


In [15]:
# Analyze trends and insights
if all_results:
    print("\n" + "="*80)
    print("KEY INSIGHTS AND ANALYSIS")
    print("="*80)
    
    # Find best and worst performing models
    best_model = comparison_df.loc[comparison_df['Final_Accuracy'].idxmax()]
    worst_model = comparison_df.loc[comparison_df['Final_Accuracy'].idxmin()]
    
    print(f"\n🏆 BEST PERFORMING MODEL:")
    print(f"  Agreement Level: {best_model['Agreement_Level']}")
    print(f"  Final Accuracy: {best_model['Final_Accuracy']:.3f}")
    print(f"  Dataset Size: {best_model['Dataset_Size']:,}")
    print(f"  Improvement: {best_model['Improvement']:.3f} ({best_model['Improvement_Percent']:+.1f}%)")
    
    print(f"\n📉 LOWEST PERFORMING MODEL:")
    print(f"  Agreement Level: {worst_model['Agreement_Level']}")
    print(f"  Final Accuracy: {worst_model['Final_Accuracy']:.3f}")
    print(f"  Dataset Size: {worst_model['Dataset_Size']:,}")
    print(f"  Improvement: {worst_model['Improvement']:.3f} ({worst_model['Improvement_Percent']:+.1f}%)")
    
    # Analyze relationship between agreement level and performance
    print(f"\n📈 TRENDS ANALYSIS:")
    print(f"  Average Final Accuracy: {comparison_df['Final_Accuracy'].mean():.3f}")
    print(f"  Average Improvement: {comparison_df['Improvement'].mean():.3f}")
    print(f"  Standard Deviation: {comparison_df['Final_Accuracy'].std():.3f}")
    
    # Check if higher agreement correlates with better performance
    agreement_mapping = {
        'sentences_50agree': 50,
        'sentences_66agree': 66,
        'sentences_75agree': 75,
        'sentences_allagree': 100
    }
    
    comparison_df['Agreement_Percent'] = comparison_df['Agreement_Level'].map(agreement_mapping)
    correlation = comparison_df['Agreement_Percent'].corr(comparison_df['Final_Accuracy'])
    
    print(f"\n🔍 CORRELATION ANALYSIS:")
    print(f"  Agreement Level vs Final Accuracy Correlation: {correlation:.3f}")
    
    if correlation > 0.5:
        print(f"  ✅ Strong positive correlation - Higher agreement improves performance")
    elif correlation > 0.2:
        print(f"  ✅ Moderate positive correlation - Higher agreement tends to improve performance")
    elif correlation > -0.2:
        print(f"  ⚠️  Weak correlation - Agreement level has minimal impact on performance")
    else:
        print(f"  ❌ Negative correlation - Unexpected result, may need investigation")
    
    print(f"\n💡 RECOMMENDATIONS:")
    print(f"  • Use {best_model['Agreement_Level']} for production deployment")
    print(f"  • Data quality vs quantity trade-off analysis completed")
    print(f"  • Consider ensemble methods if performance differences are small")
    print(f"  • Monitor real-world performance to validate these results")
    
else:
    print("❌ No analysis possible - no successful experiments")


KEY INSIGHTS AND ANALYSIS

🏆 BEST PERFORMING MODEL:
  Agreement Level: sentences_allagree
  Final Accuracy: 0.978
  Dataset Size: 2,264
  Improvement: 0.328 (+50.4%)

📉 LOWEST PERFORMING MODEL:
  Agreement Level: sentences_50agree
  Final Accuracy: 0.859
  Dataset Size: 4,846
  Improvement: 0.259 (+43.2%)

📈 TRENDS ANALYSIS:
  Average Final Accuracy: 0.922
  Average Improvement: 0.285
  Standard Deviation: 0.054

🔍 CORRELATION ANALYSIS:
  Agreement Level vs Final Accuracy Correlation: 0.939
  ✅ Strong positive correlation - Higher agreement improves performance

💡 RECOMMENDATIONS:
  • Use sentences_allagree for production deployment
  • Data quality vs quantity trade-off analysis completed
  • Consider ensemble methods if performance differences are small
  • Monitor real-world performance to validate these results


In [16]:
# Save detailed results for each agreement level
if detailed_results:
    for agreement_level, data in detailed_results.items():
        # Create detailed predictions DataFrame
        detailed_df = pd.DataFrame({
            'text': data['test_data']['text'].tolist(),
            'true_sentiment': data['y_true'],
            'predicted_sentiment': data['y_pred'],
            'correct': [t == p for t, p in zip(data['y_true'], data['y_pred'])],
            'agreement_level': agreement_level
        })
        
        # Save detailed results
        detail_filename = f"../results/detailed_predictions_{agreement_level}.csv"
        detailed_df.to_csv(detail_filename, index=False)
        print(f"📝 Detailed predictions saved: {detail_filename}")

print(f"\n✅ All results and analysis completed!")
print(f"📂 Check the ../results/ folder for all output files")

📝 Detailed predictions saved: ../results/detailed_predictions_sentences_50agree.csv
📝 Detailed predictions saved: ../results/detailed_predictions_sentences_66agree.csv
📝 Detailed predictions saved: ../results/detailed_predictions_sentences_75agree.csv
📝 Detailed predictions saved: ../results/detailed_predictions_sentences_allagree.csv

✅ All results and analysis completed!
📂 Check the ../results/ folder for all output files


In [17]:
# Final memory cleanup
clear_memory()
print("\n🧹 Final memory cleanup completed")
print("\n🎯 Experiment completed successfully!")

Clearing memory...
GPU memory allocated: 0.02 GB
GPU memory reserved: 1.53 GB

🧹 Final memory cleanup completed

🎯 Experiment completed successfully!
